# ZarrManager example

In [ ]:
# --- Imports ---
import pymif.microscope_manager as mm
import napari
import os
import time

In [ ]:
# --- Set input data path ---
main_path = "../../pymif_test_data/zarr"
# dataset_name = "viventis_5T_2C_81Z_2304Y_2304X"
dataset_name = "viventis_5T_2C_8Z_128Y_128X"
input_zarr_path = f"{main_path}/{dataset_name}.zarr"


In [ ]:
# --- Initialize manager ---
# Here, the argument chunks can be used to specify a preferred chunk (Default: None = zarr native chunks)
zarr_dataset = mm.ZarrManager(path=input_zarr_path)

In [ ]:
zarr_dataset.data

In [ ]:
# zarr_dataset.visualize_zarr()

## Create an empty copy and modify it

In [ ]:
output_zarr_path = f"{main_path}/{dataset_name}_copy_empty.zarr"

In [ ]:
# zarr_dataset.write(output_zarr_path)

In [ ]:
zarr_dataset_copy = mm.ZarrManager(path=output_zarr_path, mode="w", metadata=zarr_dataset.metadata)

## Add one part of the image

In [ ]:
zarr_dataset_copy.write_image_region(
    data = zarr_dataset.data[0][0:1,:,:4,:64,10:74].compute(),
    t = slice(0,1),
    c = slice(0,2),
    z = slice(0,4),
    y = slice(0,64),
    x = slice(10,74),
    level = 0,
    # group = "/",
)

## Add one part of the label of the membrane

First, create an empty label group

In [ ]:
labels_metadata = {
    "axes" : "tzyx",
    "size" : [tuple([s[0],s[2],s[3],s[4]]) for s in zarr_dataset.metadata["size"]],
    "chunksize" : [tuple([s[0],s[2],s[3],s[4]]) for s in zarr_dataset.metadata["chunksize"]],
    "scales" : zarr_dataset.metadata["scales"],
    "time_increment" : zarr_dataset.metadata["time_increment"],
    "time_increment_unit" : zarr_dataset.metadata["time_increment_unit"],
}
zarr_dataset_copy.create_empty_group(group_name="membranes",
                                     metadata=labels_metadata,
                                     is_label=True)

In [ ]:
zarr_dataset_copy.write_label_region(
    data = zarr_dataset.labels["membranes"][0][0:1,:4,:64,10:74].compute(),
    t = slice(0,1),
    z = slice(0,4),
    y = slice(0,64),
    x = slice(10,74),
    level = 0,
    group = "labels/membranes",
)

## Now add a new group e.g. processed and append a small part of the image after some processing

In [ ]:
labels_metadata = {
    "axes" : "tczyx",
    "size" : [tuple([s[0],1,s[2],s[3],s[4]]) for s in zarr_dataset.metadata["size"]],
    "chunksize" : zarr_dataset.metadata["chunksize"],
    "scales" : zarr_dataset.metadata["scales"],
    "time_increment" : zarr_dataset.metadata["time_increment"],
    "time_increment_unit" : zarr_dataset.metadata["time_increment_unit"],
    "units" : zarr_dataset.metadata["units"],
    "channel_names" : ["Hoechst_processed"],
    "channel_colors" : [16711935],
}

zarr_dataset_copy.create_empty_group(group_name="processed",
                                     metadata=labels_metadata,
                                     is_label=False)

In [ ]:
from skimage.filters import gaussian
array = zarr_dataset.data[0][0:1,:1,:4,:64,10:74].compute()
for i in range(array.shape[0]):
    for j in range(array.shape[1]):
        p = gaussian(array[i,j], sigma=1, preserve_range=True)
        array[i,j] = p

        

In [ ]:
import numpy as np
print(array.shape)
print(np.max(array))

In [ ]:
zarr_dataset_copy.write_image_region(
    data = np.array(array),
    t = slice(0,1),
    c = slice(0,1),
    z = slice(0,4),
    y = slice(0,64),
    x = slice(10,74),
    level = 0,
    group = "processed",
)

In [ ]:
viewer = napari.Viewer()
zarr_dataset_copy = mm.ZarrManager(path=output_zarr_path, mode="r+")
zarr_dataset_copy.visualize_zarr(viewer=viewer)